In [1]:
#Import statements for entire notebook
import pandas as pd
import numpy as np
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

In [2]:
#Load in the data
df = pd.read_csv('data\Hospital_Inpatient_Discharges_17_18_cleaned.csv')
df.head()

#Create dataframe to hold scores at each step
evaluation_df = pd.DataFrame(columns=['Model', 'Features', 'Population', 'Target Variable Normalized', 'Encoding Type', 'R2', 'MSE', 'MAE'])

C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
#Helper functions
def encode_categorical_variables(df_to_encode):
    encoded_df = df_to_encode
    categorical_columns = df_to_encode.select_dtypes(exclude='number')
    for column in categorical_columns:
        values = df_to_encode[column].unique()
        encoded_values = [i for i in range(1, len(values) + 1)]
        encoded_df[column].replace(values, encoded_values, inplace=True)
    return encoded_df

def get_scores(y_pred, y_true):
    mae = metrics.mean_absolute_error(y_true, y_pred)
    mse = metrics.mean_squared_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)
    return [r2, mse, mae]


In [4]:
#Reduce sample for training
sample_df = df.sample(30000)

#Fill na with most common values
sample_df = sample_df.fillna(sample_df.mode().iloc[0])

sample_df_heart_failure = sample_df[(sample_df['APR DRG Description'].str.startswith('Heart')) | (sample_df['APR DRG Code'] == 194)]

X = sample_df_heart_failure.loc[:, ~sample_df.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges'])]
y = sample_df_heart_failure[['Length of Stay']]

curr_columns = X.columns

X = pd.get_dummies(X)

X = X.loc[:, ~X.columns.isin(curr_columns)]

X_train_all_features, X_test_all_features, y_train_all_features, y_test_all_features = train_test_split(X, y, test_size = 0.25)


y['Length of Stay'] = y['Length of Stay'].astype('float64')

<ipython-input-4-110131b3251e>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Length of Stay'] = y['Length of Stay'].astype('float64')


In [5]:
'''tt2 = TransformedTargetRegressor(regressor=GradientBoostingRegressor(), func=np.log, inverse_func=np.exp)
tt2.fit(X_test_all_features, y_test_all_features)
preds = tt2.predict(X_test_all_features)
score = tt2.score(X_test_all_features, y_test_all_features)'''

'tt2 = TransformedTargetRegressor(regressor=GradientBoostingRegressor(), func=np.log, inverse_func=np.exp)\ntt2.fit(X_test_all_features, y_test_all_features)\npreds = tt2.predict(X_test_all_features)\nscore = tt2.score(X_test_all_features, y_test_all_features)'

In [6]:
#SVR performs well, try GridSearch with SVR
param = {'regressor__kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),'regressor__C' : [1,5,10],'regressor__degree' : [3,8],'regressor__coef0' : [0.01,10,0.5],'regressor__gamma' : ('auto','scale')},

grid_search = GridSearchCV(estimator = TransformedTargetRegressor(regressor=SVR(), func=np.log, inverse_func=np.exp), param_grid = param, 
                      cv = 3, n_jobs = -1, verbose = 2, scoring='r2')

grid_search.fit(X_train_all_features, y_train_all_features)

grid_search.best_params_

Fitting 3 folds for each of 144 candidates, totalling 432 fits


{'regressor__C': 10,
 'regressor__coef0': 0.01,
 'regressor__degree': 3,
 'regressor__gamma': 'auto',
 'regressor__kernel': 'rbf'}

In [16]:
#Reduce sample for training
sample_df = df.sample(500000)

#Fill na with most common values
sample_df = sample_df.fillna(sample_df.mode().iloc[0])

sample_df_heart_failure = sample_df[(sample_df['APR DRG Description'].str.startswith('Heart')) | (sample_df['APR DRG Code'] == 194)]

X = sample_df_heart_failure.loc[:, ~sample_df.columns.isin(['Length of Stay', 'Total Costs', 'Total Charges'])]
y = sample_df_heart_failure[['Length of Stay']]

curr_columns = X.columns

X = pd.get_dummies(X)

X = X.loc[:, ~X.columns.isin(curr_columns)]

X_train_all_features, X_test_all_features, y_train_all_features, y_test_all_features = train_test_split(X, y, test_size = 0.25)


y['Length of Stay'] = y['Length of Stay'].astype('float64')


tt2 = TransformedTargetRegressor(regressor=SVR(kernel='rbf', degree=3, gamma='scale', coef0=0.01, C=1), func=np.log, inverse_func=np.exp)
tt2.fit(X_test_all_features, y_test_all_features)
preds = tt2.predict(X_test_all_features)
score = tt2.score(X_test_all_features, y_test_all_features)
score


kf = KFold(n_splits=6, random_state=42, shuffle=True)
  # Perform cross-validation
cv_scores = cross_val_score(tt2, X, y, cv=kf, scoring='neg_mean_absolute_error')
# Append the results
print(cv_scores)
mean_cv_scores = np.mean(cv_scores)
print(mean_cv_scores)

<ipython-input-16-768b79d34470>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Length of Stay'] = y['Length of Stay'].astype('float64')


[-2.63304657 -2.61390419 -2.76328113 -2.73133746 -2.8661364  -2.76872924]
-2.7294058316289376


In [20]:
#Check other models with samller sample size to help run times
models = {'Linear Regression': LinearRegression(), 
'Ridge': Ridge(alpha=0.1), 
'Lasso': Lasso(alpha=0.1), 
'DecisionTreeRegressor':DecisionTreeRegressor(),
'RandomForestRegressor':RandomForestRegressor(),
'SVR': SVR(),
'GradientBoostingRegressor': GradientBoostingRegressor()}

scoring_methods = ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']

results = {}

for model_key in models.keys():
  model = models[model_key]
  kf = KFold(n_splits=6, random_state=42, shuffle=True)
  # Perform cross-validation
  for score_method in scoring_methods:
    cv_scores = cross_val_score(model, X, y, cv=kf, scoring=score_method)
    # Append the results
    if model_key not in results.keys():
      results[model_key] = {}
      results[model_key][score_method] = np.mean(cv_scores)
    else:
      results[model_key][score_method] = np.mean(cv_scores)

C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\Urvi\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selec

In [21]:
print(results)

{'Linear Regression': {'r2': -8.135602399913165e+21, 'neg_mean_squared_error': -3.626986457630651e+23, 'neg_mean_absolute_error': -20536754297.84912}, 'Ridge': {'r2': 0.3456191379814026, 'neg_mean_squared_error': -24.067156943043738, 'neg_mean_absolute_error': -2.897903292250486}, 'Lasso': {'r2': 0.2318663872174527, 'neg_mean_squared_error': -28.594360893639315, 'neg_mean_absolute_error': -3.0358469649582926}, 'DecisionTreeRegressor': {'r2': -0.36425215769918334, 'neg_mean_squared_error': -48.138957275597214, 'neg_mean_absolute_error': -3.7829159343878147}, 'RandomForestRegressor': {'r2': 0.2120292569434485, 'neg_mean_squared_error': -28.46006802986148, 'neg_mean_absolute_error': -3.003682210341976}, 'SVR': {'r2': 0.20375440213517515, 'neg_mean_squared_error': -29.723678796742927, 'neg_mean_absolute_error': -2.745802888101295}, 'GradientBoostingRegressor': {'r2': 0.31082987858154837, 'neg_mean_squared_error': -25.267181703268335, 'neg_mean_absolute_error': -2.890769142820242}}


In [13]:
metrics.get_scorer_names()

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'matthews_corrcoef',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_